In [1]:
# %pip install bm25s
# %pip install spacy
# %pip install -U 'spacy[cuda12x]'

In [2]:
import functions as fct
import custom_spacy as csp
import spacy
import bm25s
import numpy as np

resource module not available on Windows


Summarization with legal-pegasus

In [3]:
text = """On March 5, 2021, the Securities and Exchange Commission charged AT&T, Inc. with repeatedly violating Regulation FD, and three of its Investor Relations executives with aiding and abetting AT&T's violations, by selectively disclosing material nonpublic information to research analysts. According to the SEC's complaint, AT&T learned in March 2016 that a steeper-than-expected decline in its first quarter smartphone sales would cause AT&T's revenue to fall short of analysts' estimates for the quarter. The complaint alleges that to avoid falling short of the consensus revenue estimate for the third consecutive quarter, AT&T Investor Relations executives Christopher Womack, Michael Black, and Kent Evans made private, one-on-one phone calls to analysts at approximately 20 separate firms. On these calls, the AT&T executives allegedly disclosed AT&T's internal smartphone sales data and the impact of that data on internal revenue metrics, despite the fact that internal documents specifically informed Investor Relations personnel that AT&T's revenue and sales of smartphones were types of information generally considered "material" to AT&T investors, and therefore prohibited from selective disclosure under Regulation FD. The complaint further alleges that as a result of what they were told on these calls, the analysts substantially reduced their revenue forecasts, leading to the overall consensus revenue estimate falling to just below the level that AT&T ultimately reported to the public on April 26, 2016. The SEC's complaint, filed in federal district court in Manhattan, charges AT&T with violations of the disclosure provisions of Section 13(a) of the Securities Exchange Act of 1934 and Regulation FD thereunder, and charges Womack, Evans and Black with aiding and abetting these violations. The complaint seeks permanent injunctive relief and civil monetary penalties against each defendant. The SEC's investigation was conducted by George N. Stepaniuk, Thomas Peirce, and David Zetlin-Jones of the SEC's New York Regional Office. The SEC's litigation will be conducted by Alexander M. Vasilescu, Victor Suthammanont, and Mr. Zetlin-Jones. The case is being supervised by Sanjay Wadhwa."""

summary = fct.summarize(text, "legal-pegasus")

print(summary)

On March 5, 2021, the Securities and Exchange Commission charged AT&T, Inc. and three of its Investor Relations executives with aiding and abetting the company's violations of the antifraud provisions of Section 10(b) of the Securities Exchange Act of 1934 and Rule 10b-5 thereunder. According to the SEC's complaint, the company learned in March 2016 that a steeper-than-expected decline in its first quarter smartphone sales would cause its revenue to fall short of analysts' estimates for the quarter. The complaint alleges that to avoid falling short of the consensus revenue estimate for the third consecutive quarter, the executives made private, one-on-one phone calls to analysts at approximately 20 separate firms. On these calls, the SEC alleges that Christopher Womack, Michael Black, and Kent Evans told analysts that a steep decline in smartphone sales was a significant factor in the revenue shortfall. As a result of what they were told, the analysts substantially reduced their revenu

BM25 with custom_spacy

In [4]:
#modified spacy for segmentation of legal text
nlp = csp.custom_spacy_model()            

# Process the original text
doc = nlp(text)

# Create a corpus for BM25
corpus = [sent.text for sent in doc.sents]  

# Initialize and index BM25
retriever = bm25s.BM25(corpus=corpus)
retriever.index(bm25s.tokenize(corpus))  

# Process the summary
doc_sum = nlp(summary)

# Retrieve sentences
for sent in doc_sum.sents:
    query = sent.text
    results, scores = retriever.retrieve(bm25s.tokenize(query), k=2)
    avg_score = np.mean(scores)
    print(sent.text + " ; " + str(avg_score))


Split strings:   0%|          | 0/9 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/9 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/9 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

The SEC seeks permanent injunctive relief and civil monetary penalties against each defendant. ; 5.27659


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

The SEC investigation was conducted by George N. Stepaniuk, ; 2.5547967


In [5]:
print(text)
print(summary)

On March 5, 2021, the Securities and Exchange Commission charged AT&T, Inc. with repeatedly violating Regulation FD, and three of its Investor Relations executives with aiding and abetting AT&T's violations, by selectively disclosing material nonpublic information to research analysts. According to the SEC's complaint, AT&T learned in March 2016 that a steeper-than-expected decline in its first quarter smartphone sales would cause AT&T's revenue to fall short of analysts' estimates for the quarter. The complaint alleges that to avoid falling short of the consensus revenue estimate for the third consecutive quarter, AT&T Investor Relations executives Christopher Womack, Michael Black, and Kent Evans made private, one-on-one phone calls to analysts at approximately 20 separate firms. On these calls, the AT&T executives allegedly disclosed AT&T's internal smartphone sales data and the impact of that data on internal revenue metrics, despite the fact that internal documents specifically in

In [6]:
print(len(text), len(summary))

2206 1312


In [7]:
score = fct.rouge_evaluations(text, summary)
print(score)

   Metric  Precision    Recall  F1-Score
0  rouge1   0.919811  0.560345  0.696429
1  rouge2   0.791469  0.481268  0.598566
2  rougeL   0.768868  0.468391  0.582143


# BERT

In [1]:
import nltk
nltk.download('punkt')
from transformers import BertModel, BertTokenizer
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from nltk.tokenize import sent_tokenize
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Charger le modèle et le tokenizer BERT pré-entraîné
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

c:\Users\Nicolas\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nicolas\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
# Exemple de texte juridique
document = """
La loi garantit les droits et libertés fondamentaux de chaque citoyen.
Le juge a rendu son verdict en se basant sur la loi de 2023.
Les droits des citoyens doivent être respectés dans toute procédure judiciaire.
La constitution de 1958 est la base du système juridique actuel.
Chaque individu a le droit à un procès équitable selon les lois en vigueur.
"""

In [4]:
# Tokenisation en phrases
sentences = sent_tokenize(document)

# Fonction pour obtenir les embeddings de phrases avec BERT
def get_sentence_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
        outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy())  # Moyenne des embeddings
    return np.array(embeddings)

# Obtenir les embeddings des phrases
sentence_embeddings = get_sentence_embeddings(sentences)

# K-Means

In [5]:
# Utiliser K-Means pour regrouper des phrases similaires
n_clusters = 3  # Nombre de clusters souhaités (peut être ajusté)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(sentence_embeddings)

# Étiquettes des clusters
labels = kmeans.labels_

c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [6]:
cluster = {}
for i in range(n_clusters):
    print(f"\nCluster {i+1}:")
    cluster[i] = []  # Initialiser une liste pour chaque cluster
    for j, sentence in enumerate(sentences):
        if labels[j] == i:
            print(f"- {sentence}")
            # Ajouter 'sentence' au cluster 'i'
            cluster[i].append(sentence)
    # Afficher les clés du dictionnaire 'cluster'
print(cluster.keys())


Cluster 1:
- 
La loi garantit les droits et libertés fondamentaux de chaque citoyen.
- Les droits des citoyens doivent être respectés dans toute procédure judiciaire.

Cluster 2:
- La constitution de 1958 est la base du système juridique actuel.

Cluster 3:
- Le juge a rendu son verdict en se basant sur la loi de 2023.
- Chaque individu a le droit à un procès équitable selon les lois en vigueur.
dict_keys([0, 1, 2])


In [7]:
# Calculer le score de silhouette pour évaluer la qualité du clustering
silhouette_avg = silhouette_score(sentence_embeddings, labels)
print(f"\nSilhouette Score: {silhouette_avg}")


Silhouette Score: 0.12059376388788223


# BM25

In [8]:
# Installer la bibliothèque si nécessaire
%pip install rank-bm25

  Obtaining dependency information for rank-bm25 from https://files.pythonhosted.org/packages/2a/21/f691fb2613100a62b3fa91e9988c991e9ca5b89ea31c0d3152a3210344f9/rank_bm25-0.2.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [9]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Exemple de documents (textes juridiques)
documents = [
    "a cat is a feline and likes to purr",
    "a dog is the human's best friend and loves to play",
    "a bird is a beautiful animal that can fly",
    "a fish is a creature that lives in water and swims"
]

In [11]:
# # Tokenisation des documents
# tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]

# # Initialiser le modèle BM25
# bm25 = BM25Okapi(tokenized_docs)

# Dictionnaire pour stocker les documents tokenisés par cluster
tokenized_clusters = {}

# Tokenisation des documents pour chaque cluster
for i, sentences in cluster.items():
    tokenized_clusters[i] = [word_tokenize(sentence.lower()) for sentence in sentences]

# Initialiser un modèle BM25 pour chaque cluster
bm25_models = {}
for i, tokenized_docs in tokenized_clusters.items():
    bm25_models[i] = BM25Okapi(tokenized_docs)

In [12]:
# Exemple de requête
query = "citoyens"
tokenized_query = word_tokenize(query.lower())

In [13]:
# Calcul des scores BM25 pour tous les clusters
for cluster_id, bm25 in bm25_models.items():
    # Calcul des scores pour la requête dans chaque cluster
    scores = bm25.get_scores(tokenized_query)
    print(f"Scores pour la requête dans le cluster {cluster_id}: {scores}")

Scores pour la requête dans le cluster 0: [0. 0.]
Scores pour la requête dans le cluster 1: [0.]
Scores pour la requête dans le cluster 2: [0. 0.]


In [14]:
# # Afficher les résultats
# for i, score in enumerate(scores):
#     print(f"Document {i + 1} Score: {score}")

In [15]:
# Afficher les tokens de chaque document dans le cluster pour vérifier la tokenisation
for cluster_id, tokenized_docs in tokenized_clusters.items():
    print(f"\nCluster {cluster_id}:")
    for i, doc_tokens in enumerate(tokenized_docs):
        print(f"Document {i + 1} tokens: {doc_tokens}")

# Exemple de requête
query = "Les droits et libertés des citoyens"
tokenized_query = word_tokenize(query.lower())
print(f"\nTokens de la requête: {tokenized_query}")

# Calcul des scores BM25 pour tous les clusters
for cluster_id, bm25 in bm25_models.items():
    # Calcul des scores pour la requête dans chaque cluster
    scores = bm25.get_scores(tokenized_query)
    print(f"Scores pour la requête dans le cluster {cluster_id}: {scores}")


Cluster 0:
Document 1 tokens: ['la', 'loi', 'garantit', 'les', 'droits', 'et', 'libertés', 'fondamentaux', 'de', 'chaque', 'citoyen', '.']
Document 2 tokens: ['les', 'droits', 'des', 'citoyens', 'doivent', 'être', 'respectés', 'dans', 'toute', 'procédure', 'judiciaire', '.']

Cluster 1:
Document 1 tokens: ['la', 'constitution', 'de', '1958', 'est', 'la', 'base', 'du', 'système', 'juridique', 'actuel', '.']

Cluster 2:
Document 1 tokens: ['le', 'juge', 'a', 'rendu', 'son', 'verdict', 'en', 'se', 'basant', 'sur', 'la', 'loi', 'de', '2023', '.']
Document 2 tokens: ['chaque', 'individu', 'a', 'le', 'droit', 'à', 'un', 'procès', 'équitable', 'selon', 'les', 'lois', 'en', 'vigueur', '.']

Tokens de la requête: ['les', 'droits', 'et', 'libertés', 'des', 'citoyens']
Scores pour la requête dans le cluster 0: [-0.11495985 -0.11495985]
Scores pour la requête dans le cluster 1: [0.]
Scores pour la requête dans le cluster 2: [0. 0.]
